#  Поиск аномалий (Anomaly Detection)

Поиска аномалий решает две задачи:
- детекция выбросов (outlier detection): нахождение атипичных точек, которые будут мешать алгоритму и потому требуют дополнительного внимания;
- обнаружение новизны (novelty detection): нахождение точек, которые отделяются от основных данных, и сигнализируют о чем-то новом, что мы еще не знаем

Мы боремся с выбросами, если удаляем или обрабатываем крайние значения. Их надо найти, чтобы при обучении алгоритм не концентрировался на них, а искал закономерности в основном наборе точек.

Мы ищем новизну, когда сканируем объекты на схожесть остальным: если объект оказался отличен, мы обнаружили новое. 

Иногда задачи обнаружения аномалий маскируются как задачи обучения с учителем. При условии, что есть частичная разметка данных на аномальные и неаномальные случаи. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

import warnings
warnings.filterwarnings('ignore')

Перед нам данные о температурном режиме в офисе. Разметки, где аномалия, а где норма - нет.

In [ ]:
data = pd.read_csv("ambient_temperature_system_failure.csv")
data.head()

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['value'] = (data['value'] - 32) * 5/9
data.plot(x='timestamp', y='value')

## Статистические подходы

### Процентили и бокс-плот

In [ ]:
sns.boxplot(data['value'])

<img src="https://i.ytimg.com/vi/0Bg6z9BZSMc/maxresdefault.jpg">

In [ ]:
LQ = data['value'].quantile(0.25)
UQ = data['value'].quantile(0.75)
print(data.value.min(), data.value.max())
print(LQ, UQ)
IQR = UQ - LQ
lower_bound = max(data.value.min(), LQ - 1.5*IQR)
upper_bound = min(data.value.max(), UQ + 1.5*IQR)
print(lower_bound)
print(upper_bound)

### Z-value

$$ Z_i = \frac{|x - \mu|}{\sigma} $$

In [ ]:
mu = data.value.mean()
sigma = data.value.std()
z_values = [np.abs(x - mu) / sigma for x in data.value]

In [ ]:
sns.histplot(z_values)

## ML. Эллипсоидальная аппроксимация данных

Основная предпосылка - данные должны быть нормально распределены. Кроме того, нам нужно предположить, какая доля аномальных объектов в датасете - это гиперпараметр *contamination*

Идея метода в том, что вокруг точек создается воображаемое эллиптическое облако. Это облако, т.н. Elliptic Envelope, - зона нормальных точек. А те, кто выпали из этого "конверта" - называются аномальными.

In [ ]:
data['value'].hist()

In [ ]:
outliers_fraction = 0.01

In [ ]:
model = EllipticEnvelope(contamination=outliers_fraction)
array = data['value'].values.reshape(-1,1)

model.fit(array)

data['deviation'] = model.decision_function(array)
data['anomaly_elptc'] = model.predict(array)
data['anomaly_elptc'] = data['anomaly_elptc'].map( {1: 0, -1: 1} )

In [ ]:
data['anomaly_elptc'].value_counts()

In [ ]:
a = data[data['anomaly_elptc'] == 1][['timestamp', 'value']]

plt.plot(data['timestamp'], data['value'], color='blue')
plt.scatter(a['timestamp'], a['value'], color='red')
plt.xticks(rotation=90)
plt.show()

## Препроцессинг

In [ ]:
data['hours'] = data['timestamp'].dt.hour
data['daylight'] = ((data['hours'] >= 7) & (data['hours'] <= 22)).astype(int)
data['DayOfTheWeek'] = data['timestamp'].dt.dayofweek
data['WeekDay'] = (data['DayOfTheWeek'] < 5).astype(int)

In [ ]:
data_flt = data[['value', 'hours', 'daylight', 'DayOfTheWeek', 'WeekDay']]
min_max_scaler = StandardScaler()
np_scaled = min_max_scaler.fit_transform(data_flt)
data_flt = pd.DataFrame(np_scaled)


# знаменитый PCA
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_flt)
print(pca.explained_variance_ratio_)


# повторное шкалирование двух фичей
min_max_scaler = StandardScaler()
np_scaled = min_max_scaler.fit_transform(data_pca)
data_scld = pd.DataFrame(np_scaled)

## ML. Кластерные методы

In [ ]:
inertia = []
plt.figure(figsize= (15,8))
for n_c in range(2,20):
    k_means = KMeans(n_clusters = n_c)
    k_means = k_means.fit(data_scld)
    inertia.append(np.sqrt(k_means.inertia_))

plt.show()

fig, ax = plt.subplots(figsize=(10,6))
plt.plot(range(2, 20), inertia, marker='s');
plt.xlabel('$k$')
plt.ylabel('$J(C_k)$');

In [ ]:
k_means = KMeans(n_clusters = 4)
k_means = k_means.fit(data_scld)
data['cluster'] = k_means.predict(data_scld)
data['cluster'].value_counts()

In [ ]:
data['pc1'] = data_pca[:, 0]
data['pc2'] = data_pca[:, 1]

In [ ]:
plt.figure(figsize=(15, 8))
sns.scatterplot(data['pc1'], data['pc2'], c=data['cluster'])

In [ ]:
def getDistanceByPoint(data, model):
    distance = pd.Series(data.shape[0])
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]]
        distance[i] = np.linalg.norm(Xa-Xb)
    return distance

In [ ]:
distance = getDistanceByPoint(data_scld, k_means)
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()


data['anomaly_kmeans'] = (distance >= threshold).astype(int)

In [ ]:
data['anomaly_kmeans'].value_counts()

In [ ]:
plt.figure(figsize=(15, 8))
sns.scatterplot(data['pc1'], data['pc2'], c=data['anomaly_kmeans'])

In [ ]:
a = data[data['anomaly_kmeans'] == 1][['timestamp', 'value']]

plt.plot(data['timestamp'], data['value'], color='blue')
plt.scatter(a['timestamp'], a['value'], color='red')
plt.xticks(rotation=90)
plt.show()

## ML. Local Outlier Factor

Основан на наблюдении, что нормальные наблюдения имеют тенденцию скапливаться

- вводится показатель локальной плотности, обратно пропорциональный средним расстоянием до  𝑘  ближайших соседей
- попарно сравнивается с показателями соседей
- вычисляется отношение локальной аномальности

In [ ]:
# n_neighbors = 20
model =  LocalOutlierFactor()
data['anomaly_lof'] = pd.Series(model.fit_predict(data_flt))
data['anomaly_lof'] = data['anomaly_lof'].map( {1: 0, -1: 1} )
print(data['anomaly_svm'].value_counts())

In [ ]:
a = data[data['anomaly_lof'] == 1][['timestamp', 'value']]

plt.plot(data['timestamp'], data['value'], color='blue')
plt.scatter(a['timestamp'], a['value'], color='red')
plt.xticks(rotation=90)
plt.show()

In [ ]:
a = data[data['anomaly_lof'] == 0]['value']
b = data[data['anomaly_lof'] == 1]['value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
plt.legend()
plt.show()

- подвержен проблеме "проклятия размерности", тк основан на расстояниях
- не может отличить скопления аномалий от нормальных точек

## ML. One Class SVM

<img src='https://avatars.mds.yandex.net/get-zen_doc/1692094/pub_5d444f9ca660d700aeec498a_5d445006c0dcf200adbc8c0e/scale_1200'>

<img src="https://avatars.mds.yandex.net/get-zen_doc/56585/pub_5d444f9ca660d700aeec498a_5d445016f0d4f400ae4b367e/scale_1200">

In [ ]:
# nu - гиперпараметр, регулирующй долю аутлайеров
model =  OneClassSVM(nu=outliers_fraction)
model.fit(data_flt)


data['anomaly_svm'] = pd.Series(model.predict(data_flt))
data['anomaly_svm'] = data['anomaly_svm'].map( {1: 0, -1: 1} )
print(data['anomaly_svm'].value_counts())

In [ ]:
a = data[data['anomaly_svm'] == 1][['timestamp', 'value']]

plt.plot(data['timestamp'], data['value'], color='blue')
plt.scatter(a['timestamp'], a['value'], color='red')
plt.xticks(rotation=90)
plt.show()

In [ ]:
a = data[data['anomaly_svm'] == 0]['value']
b = data[data['anomaly_svm'] == 1]['value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
plt.legend()
plt.show()

- вычислительно затратен и плохо масштабируется

## ML. Isolation Forest

<img src="https://1.bp.blogspot.com/-_Voh6L1tL8k/Xp0rMEjbt8I/AAAAAAAAAgg/hXY_aC5erAoRUdeaS92Rp-gjsVI6l02XwCLcBGAsYHQ/s1600/isolation%2Bforest.png">

<img src='https://blog.faradars.org/wp-content/uploads/2020/05/partition-of-Isolation-forest.png'>

In [ ]:
#  учим на data_flt - отшкалированных данных с признакми
model =  IsolationForest(contamination = outliers_fraction)
model.fit(data_flt)


data['anomay_if'] = pd.Series(model.predict(data_flt))
data['anomay_if'] = data['anomay_if'].map( {1: 0, -1: 1} )
print(data['anomay_if'].value_counts())

In [ ]:
a = data[data['anomay_if'] == 1][['timestamp', 'value']]

plt.plot(data['timestamp'], data['value'], color='blue')
plt.scatter(a['timestamp'], a['value'], color='red')
plt.xticks(rotation=90)
plt.show()

Минусы: 
- не различает скопления аномалий